Define data. The index_col = 0 indicates that the row names are in the first column.  The function read_excel() assumes that the first row is column names.  price_data.index gives the row names of the data frame price_data.

In [1]:
import pandas as pd

hours_data = pd.read_excel("Manufacturing-Homework.xlsx", sheet_name="Hours", index_col=0)
P = list(hours_data.index)
M = list(hours_data)
machine_hours = hours_data.stack().to_dict()
print(machine_hours)

demand_data = pd.read_excel("Manufacturing-Homework.xlsx", sheet_name="Demand", index_col=0)
#P = list(demand_data.index)
#^do i need this? (since it was identified in previous sheet)
T = list(demand_data)
demand = demand_data.stack().to_dict()
print(demand)

price_data = pd.read_excel("Manufacturing-Homework.xlsx", sheet_name="Price", index_col=0)
#P = list(demand_data.index)
#T = list(price_data)
price = price_data.stack().to_dict()
print(price)



{('Product 1', 'Machine 1'): 0.4, ('Product 1', 'Machine 2'): 0.3, ('Product 2', 'Machine 1'): 0.7, ('Product 2', 'Machine 2'): 0.4}
{('Product 1', 'Month 1'): 1000, ('Product 1', 'Month 2'): 1900, ('Product 2', 'Month 1'): 1400, ('Product 2', 'Month 2'): 1300}
{('Product 1', 'Month 1'): 55, ('Product 1', 'Month 2'): 12, ('Product 2', 'Month 1'): 65, ('Product 2', 'Month 2'): 32}


Import Pyomo and declare a model object.

In [2]:
from pyomo.environ import *

model = ConcreteModel()

Declare nonnegative variables for amount produced each month.

In [3]:
try: 
    model.del_component(model.x)
    model.del_component(model.x_index)
    model.del_component(model.x_index_0)
    model.del_component(model.x_index_1)
    model.del_component(model.x_index_2)
    model.del_component(model.x_index_index_0)
except:
    print("Defining variables for first time")
model.x = Var(P,M,T,domain=NonNegativeReals)
model.pprint()

Defining variables for first time


5 Set Declarations
    x_index : Dim=0, Dimen=3, Size=8, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index_0 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Product 1', 'Product 2']
    x_index_1 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Machine 1', 'Machine 2']
    x_index_2 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Month 1', 'Month 2']
    x_index_index_0 : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual

1 Var Declarations
    x : Size=8, Index=x_index
        Key                                   : Lower : Value : Upper : Fixed : Stale : Domain
        ('Product 1', 'Machine 1', 'Month 1') :     0 :  None :  None : False :  True : NonNegativeReals
        ('Product 1', 'Machine 1', 'Month 2') :     0 :  None :  None : False :  True : NonNegativeReals
        ('Product 1', 'Machine 2', 'Month 1') :     0 :  None :  None : False :  True : N

Define nonnegative variables for inventory.

In [4]:
try:
    model.del_component(model.Profit)
except:
    print("Defining objective Profit for the first time")
model.Profit = Objective(
    expr = sum(sum(sum((price[i,k])*model.x[i,j,k] for i in P) for j in M) for k in T),  sense=maximize)
model.pprint()

Defining objective Profit for the first time
5 Set Declarations
    x_index : Dim=0, Dimen=3, Size=8, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index_0 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Product 1', 'Product 2']
    x_index_1 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Machine 1', 'Machine 2']
    x_index_2 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Month 1', 'Month 2']
    x_index_index_0 : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual

1 Var Declarations
    x : Size=8, Index=x_index
        Key                                   : Lower : Value : Upper : Fixed : Stale : Domain
        ('Product 1', 'Machine 1', 'Month 1') :     0 :  None :  None : False :  True : NonNegativeReals
        ('Product 1', 'Machine 1', 'Month 2') :     0 :  None :  None : False :  True : NonNegativeReals
        ('Product 1', 'Machine 2', 'Month 1'

Define the objective function.

In [5]:
try:
    model.del_component(model.DemandConstraint)
    model.del_component(model.DemandConstraint_index)
except:
    print("Defining demand constraint for the first time")
model.DemandConstraint = ConstraintList()


Defining demand constraint for the first time


In [6]:
for i in P:
    for k in T:
        model.DemandConstraint.add(sum(model.x[i,j,k] for j in M) >= demand[i,k])
model.pprint()

6 Set Declarations
    DemandConstraint_index : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=None
        [1, 2, 3, 4]
    x_index : Dim=0, Dimen=3, Size=8, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index_0 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Product 1', 'Product 2']
    x_index_1 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Machine 1', 'Machine 2']
    x_index_2 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Month 1', 'Month 2']
    x_index_index_0 : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual

1 Var Declarations
    x : Size=8, Index=x_index
        Key                                   : Lower : Value : Upper : Fixed : Stale : Domain
        ('Product 1', 'Machine 1', 'Month 1') :     0 :  None :  None : False :  True : NonNegativeReals
        ('Product 1', 'Machine 1', 'Month 2') :     0 :  None :  None : False : 

Specify the inventory balance constraints.

In [7]:
try:
    model.del_component(model.HoursConstraint)
    model.del_component(model.HoursConstraint_index)
except:
    print("Defining hours constraint for the first time")
model.HoursConstraint = ConstraintList()


Defining hours constraint for the first time


In [8]:
for j in M:
    for k in T:
        model.DemandConstraint.add(sum(machine_hours[i,j]*model.x[i,j,k] for i in P) >= 500)
model.pprint()

7 Set Declarations
    DemandConstraint_index : Dim=0, Dimen=1, Size=8, Domain=None, Ordered=False, Bounds=None
        [1, 2, 3, 4, 5, 6, 7, 8]
    HoursConstraint_index : Dim=0, Dimen=1, Size=0, Domain=None, Ordered=False, Bounds=None
        []
    x_index : Dim=0, Dimen=3, Size=8, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index_0 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Product 1', 'Product 2']
    x_index_1 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Machine 1', 'Machine 2']
    x_index_2 : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['Month 1', 'Month 2']
    x_index_index_0 : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual

1 Var Declarations
    x : Size=8, Index=x_index
        Key                                   : Lower : Value : Upper : Fixed : Stale : Domain
        ('Product 1', 'Machine 1', 'Month 1') :     0 :  None :  None 

Specify solver and solve.


In [9]:
solver = SolverFactory('glpk')
status = solver.solve(model)

Get status.

In [10]:
print('Status = %s' % status)

Status = 
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 9
  Number of nonzeros: 17
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.02299666404724121



Get solution and objective function value.

In [19]:
for i in P:
    for k in T:
        for j in M:
            print('%s = %f' % (model.x[i,j,k], value(model.x[i,j,k])))
print("Objective = %f" % value(model.Profit))

KeyError: "Index '('Product 1', 'Month 1')' is not valid for indexed component 'x'"

ValueError: No value for uninitialized NumericValue object x[Product 1,Machine 1,Month 1]

The optimal solution is to produce 1150 units in month 1 and 1700 in month 3.  Hold 650 in inventory after month 1, and hold 700 in inventory after month 3.  The total cost is $152500.